
# BIN to TIF Conversion (Latitude, Longitude, Value)

This notebook converts three **binary (.bin)** files containing **latitude, longitude, and values** 
into a single georeferenced **.TIF** file.  

---


## Import needed libraries

In [1]:

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tifffile import imwrite


## General parameters

In [25]:

# Input directory containing BIN files
input_dir = Path("data/bin_files")

# Output directory for TIF files
output_dir = Path("data/tif_files")
output_dir.mkdir(parents=True, exist_ok=True)

# Expected shape of the data (to be adapted to your dataset)
image_shape = (512, 512)  # Example: 512x512 pixels

# File names 
lat_file = input_dir / "latitude.bin"
lon_file = input_dir / "longitude.bin"
val_file = input_dir / "values.bin"

# Output TIF file
output_file = output_dir / "output_image.tif"


## Functions

In [20]:

def load_bin_file(bin_path: Path, dtype=np.float32) -> np.ndarray:
    """
    Load a binary file into a numpy array.

    Args:
        bin_path (Path): Path to the .bin file
        dtype (np.dtype): Data type of the binary file

    Returns:
        np.ndarray: 1D numpy array containing the file data
    """
    return np.fromfile(bin_path, dtype=dtype)


def validate_inputs(lat: np.ndarray, lon: np.ndarray, val: np.ndarray) -> None:
    """
    Validate that latitude, longitude, and value arrays have the same length.

    Args:
        lat, lon, val (np.ndarray): Arrays to validate

    Raises:
        ValueError: If the arrays do not have the same length
    """
    if not (len(lat) == len(lon) == len(val)):
        raise ValueError(f"Input arrays must have the same length: "
                         f"lat={len(lat)}, lon={len(lon)}, val={len(val)}")


def reshape_array(array: np.ndarray, shape: tuple[int, int]) -> np.ndarray:
    """
    Reshape a flat array into a 2D image.

    Args:
        array (np.ndarray): 1D array
        shape (tuple[int,int]): Target shape (height, width)

    Returns:
        np.ndarray: 2D reshaped array
    """
    return array.reshape(shape)


def save_as_tif(lat: np.ndarray, lon: np.ndarray, val: np.ndarray, tif_path: Path) -> None:
    """
    Save latitude, longitude, and values as a 3-channel TIF.

    Args:
        lat, lon, val (np.ndarray): 2D arrays of the same shape
        tif_path (Path): Path to output TIF file
    """
    # Stack arrays into a 3D volume (channels last)
    stacked = np.stack([lat, lon, val], axis=-1)
    imwrite(tif_path, stacked.astype(np.float32))
    print(f"✅ TIF saved at {tif_path}")


## Conversion pipeline

In [ ]:

# Load binary files
lat = load_bin_file(lat_file, dtype=np.float32)
lon = load_bin_file(lon_file, dtype=np.float32)
val = load_bin_file(val_file, dtype=np.float32)

# Validate consistency
validate_inputs(lat, lon, val)

# Reshape arrays
lat_2d = reshape_array(lat, image_shape)
lon_2d = reshape_array(lon, image_shape)
val_2d = reshape_array(val, image_shape)

# Save stacked TIF
save_as_tif(lat_2d, lon_2d, val_2d, output_file)


## Basic tests and validation

In [ ]:

print("Lat range:", lat.min(), "to", lat.max())
print("Lon range:", lon.min(), "to", lon.max())
print("Val range:", val.min(), "to", val.max())

print("Shapes:", lat_2d.shape, lon_2d.shape, val_2d.shape)


## Example visualization

In [ ]:

# Display the value array as grayscale
plt.imshow(val_2d, cmap="gray")
plt.title("Value Map")
plt.colorbar(label="Value")
plt.axis("off")
plt.show()



---

## 📌 Notes
- Adapt `image_shape` and `dtype` according to your data.  
- The three `.bin` files must have the same length.  
- The output **TIF** stores latitude, longitude, and values in three channels.  
- For large datasets, consider using memory mapping (`np.memmap`) instead of loading into memory.

---
